In [41]:
import pandas as pd
import config
import os
import sqlalchemy

In [42]:

_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.admin}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True})

In [43]:
directories = [x[0] for x in os.walk('/Users/schlinkertc/Flatiron/projects/classification/txt_files')][1:]
file_names = [x[2] for x in os.walk(directories[0])][0]

tag_paths = sorted([x+'/'+file_names[0] for x in directories],reverse=True)
pre_paths = sorted([x+'/'+file_names[1] for x in directories],reverse=True)
sub_paths = sorted([x+'/'+file_names[2] for x in directories],reverse=True)
num_paths = sorted([x+'/'+file_names[4] for x in directories],reverse=True)

#Dataframes for results from 2018-2019

tags = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='unicode_escape') for x in tag_paths[:7]
          ],axis=0
         )

presentations = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='utf-8') for x in pre_paths[:7]
          ],axis=0
         )

numbers = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='unicode_escape') for x in num_paths[:7]
          ],axis=0
         )

submissions = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='utf-8') for x in sub_paths[:7]
          ],axis=0
         )


In [44]:
submissions = submissions[submissions['form']=='10-K']

In [45]:
numbers = numbers[numbers['adsh'].isin(submissions['adsh'])]
presentations = presentations[presentations['adsh'].isin(submissions['adsh'])]

In [46]:
tags.drop(index=tags[tags['custom']==1].index,inplace=True)

In [70]:
ISnumbers = numbers.copy(deep=True)
ISnumbers = ISnumbers.join(presentations['stmt'],)
#ISnumbers = ISnumbers[ISnumbers['stmt']=='IS']

# ISdesired_values = [
#     'Revenues',
#     'GrossProfit',
#     'OperatingIncomeLoss',
#     'NetIncomeLoss',
#     'ProfitLoss',
#     'OperatingExpenses',    
# ]

ISdesired_values = ['NetIncomeLoss',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'OperatingIncomeLoss',
 'Revenues',
 'IncomeTaxExpenseBenefit',
 'CashAndCashEquivalentsAtCarryingValue',
 'StockholdersEquity',
 'GrossProfit',
 'NetCashProvidedByUsedInOperatingActivities',
 'ProfitLoss',
 'NetCashProvidedByUsedInFinancingActivities',
 'InterestExpense',
 'WeightedAverageNumberOfSharesOutstandingBasic',
 'Assets',
 'WeightedAverageNumberOfDilutedSharesOutstanding',
 'NetCashProvidedByUsedInInvestingActivities',
 'LiabilitiesAndStockholdersEquity',
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'ShareBasedCompensation',
 'RetainedEarningsAccumulatedDeficit',
 'CashAndCashEquivalentsPeriodIncreaseDecrease',
 'ComprehensiveIncomeNetOfTax',
 'OperatingExpenses',
 'EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate']

ISnumbers = ISnumbers[ISnumbers['tag'].isin(ISdesired_values)]

In [71]:
ISnumbers.shape

(1030362, 10)

In [72]:
ISdf = ISnumbers.merge(submissions[['adsh','name','cik','sic','form','fy']],
               how='left',
               left_on='adsh',
               right_on='adsh')

In [73]:
ISdf.set_index(['cik'],inplace=True)
ISdf.drop(columns=['name','footnote','ddate','version','form','coreg','uom','stmt'],inplace=True)

In [74]:
ISdf = ISdf.loc[(ISdf['qtrs']==4) & (ISdf['fy'].astype('int')==2018)]

In [75]:
ISdf.drop_duplicates(inplace=True)

In [76]:
ISdf.drop(columns=['fy','qtrs'],inplace=True)

In [77]:
ISdf['manufacturing'] = ISdf['sic'].map(lambda x: 2000<x<4000)
ISdf.drop(columns='sic',inplace=True)

In [78]:
#ISdf.dropna(inplace=True)

In [79]:
piv = ISdf.reset_index().pivot_table(index=['cik','adsh'],columns='tag',values='value')

In [80]:
numbers['tag'].value_counts().sort_values(ascending=False).index[:25].tolist()

['NetIncomeLoss',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'OperatingIncomeLoss',
 'Revenues',
 'IncomeTaxExpenseBenefit',
 'CashAndCashEquivalentsAtCarryingValue',
 'StockholdersEquity',
 'GrossProfit',
 'NetCashProvidedByUsedInOperatingActivities',
 'ProfitLoss',
 'NetCashProvidedByUsedInFinancingActivities',
 'InterestExpense',
 'WeightedAverageNumberOfSharesOutstandingBasic',
 'Assets',
 'WeightedAverageNumberOfDilutedSharesOutstanding',
 'NetCashProvidedByUsedInInvestingActivities',
 'LiabilitiesAndStockholdersEquity',
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'ShareBasedCompensation',
 'RetainedEarningsAccumulatedDeficit',
 'CashAndCashEquivalentsPeriodIncreaseDecrease',
 'ComprehensiveIncomeNetOfTax',
 'OperatingExpenses',
 'EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate']

In [81]:
len(ISdf.reset_index()['cik'].unique())

5592

In [84]:
piv.isna().sum(axis=0).sort_values()

tag
NetCashProvidedByUsedInOperatingActivities                                                      359
NetIncomeLoss                                                                                   379
NetCashProvidedByUsedInFinancingActivities                                                      385
NetCashProvidedByUsedInInvestingActivities                                                      711
IncomeTaxExpenseBenefit                                                                        1187
OperatingIncomeLoss                                                                            1309
ShareBasedCompensation                                                                         1591
EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate                            1629
InterestExpense                                                                                2102
ComprehensiveIncomeNetOfTax                                                                    2

In [89]:
piv['Income/OperationsCash'] = piv['NetCashProvidedByUsedInOperatingActivities']/piv['NetIncomeLoss']
piv['Income/FinancingCash'] = piv['NetCashProvidedByUsedInFinancingActivities']/piv['NetIncomeLoss']
piv['Income/Investing'] = piv['NetCashProvidedByUsedInInvestingActivities']/piv['NetIncomeLoss']

In [91]:
df = piv[['Income/OperationsCash','Income/FinancingCash','Income/Investing']].reset_index()

In [94]:
df = df.merge(ISdf['manufacturing'],
        how='right',
        left_on='cik',
        right_on='cik')

In [98]:
df = df.drop_duplicates().reset_index()

In [100]:
df['manufacturing'] = df['manufacturing'].astype(int)

In [101]:
df

,index,cik,adsh,Income/OperationsCash,Income/FinancingCash,Income/Investing,manufacturing
0,0,1750,0001047469-18-004978,0.986644,-0.338063,-0.750417,1
1,43,1800,0001047469-19-000624,3.550766,-1.065960,-2.643581,1
2,90,1961,0001264931-19-000040,2.477989,-1.202215,-5.056292,0
3,109,2034,0001144204-18-051414,-0.660066,-0.674697,1.091022,0
4,159,2098,0001026608-19-000011,1.073440,0.792700,-2.035669,1
...,...,...,...,...,...,...,...
5603,194687,1756180,0001683168-19-001042,-0.500135,0.531358,-0.038211,0
5604,194721,1757067,0001551163-19-000081,0.994917,-0.994917,NaN,0
5605,194726,1757898,0001757898-19-000005,2.016116,-1.302362,-0.739516,1
5606,194783,1758009,0001213900-19-005062,0.156186,-0.271230,0.000455,0
